In [1]:
#|hide
#|eval: false
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
#|hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
#|default_exp loggers.core

In [1]:
#|export
# Python native modules
import os,typing
# Third party libs
from fastcore.all import *
from torch.multiprocessing import Pool,Process,set_start_method,Manager,get_start_method,Queue
import torchdata.datapipes as dp
from fastprogress.fastprogress import *
# Local modules
from fastrl.pipes.core import *

# Loggers Core
> Core Utilties for logging in fastrl using Callbacks and DataPipes

In [2]:
#|export
class LoggerBase(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe=None):
        self.source_datapipe = source_datapipe
        self.main_queue = self.initialize_queue()
        
    def initialize_queue(self):
        "If the start method is `spawn` then the queue will need to be managed using a Manager."
        if get_start_method()=='spawn':
            self.manager = Manager()
            return self.manager.Queue()
        else:
            return Queue()
        
        
    def connect_source_datapipe(self,pipe):
        self.source_datapipe = pipe
        return self
        

The `LoggerBase` class outlines simply the `main_queue`. It works in combo with `LogCollector` datapipe which will add to the `main_queue` even across processes. 

In [6]:
#|export
class LogCollector(dp.iter.IterDataPipe):
    def __init__(self,
         source_datapipe, # The parent datapipe, likely the one to collect metrics from
         logger_bases:List[LoggerBase] # `LoggerBase`s that we want to send metrics to
        ):
        self.source_datapipe = source_datapipe
        self.main_queues = [o.main_queue for o in logger_bases]
        
    def __iter__(self): raise NotImplementedError

Notes:

User can init multiple different logger bases if they want

We then can manually add Collectors, custom for certain pipes such as for collecting rewards. This can be put in 
full on pipelines even across workers.

The manual process might not be desireable though, so I'm thinking of revising the `add_cbs_to_pipes` where
the pipe itself literally says if it should go before, after, exclude from.

This seems... easier than defining a callback, and keeps a level of flatness to the api.





In [7]:
#|export
class Record(typing.NamedTuple):
    name:str
    value:typing.Any

In [8]:
#|export
class ProgressBarLogger(LoggerBase):
    def __init__(self,
                 # This does not need to be immediately set since we need the `LogCollectors` to 
                 # first be able to reference its queues.
                 source_datapipe=None, 
                 # For automatic pipe attaching, we can designate which pipe this should be
                 # referneced for information on which epoch we are on
                 epoch_on_pipe:dp.iter.IterDataPipe=None,
                 # For automatic pipe attaching, we can designate which pipe this should be
                 # referneced for information on which batch we are on
                 batch_on_pipe:dp.iter.IterDataPipe=None
                ):
        self.source_datapipe = source_datapipe
        self.main_queue = self.initialize_queue()
        self.epoch_on_pipe = epoch_on_pipe
        self.batch_on_pipe = batch_on_pipe
        
        self.collector_keys = None
        self.attached_collectors = None
    
    def dequeue(self): 
        while not self.main_queue.empty(): yield self.main_queue.get()
        
    def __iter__(self):
        epocher = find_pipe_instance(self,self.epoch_on_pipe)
        batcher = find_pipe_instance(self,self.batch_on_pipe)
        mbar = master_bar(range(epocher.epochs)) 
        pbar = progress_bar(range(batcher.batches),parent=mbar,leave=False)

        mbar.update(0)
        for i,record in enumerate(self.source_datapipe):
            if i==0:
                self.attached_collectors = {o.name:o.value for o in self.dequeue()}
                mbar.write(self.attached_collectors, table=True)
                self.collector_keys = list(self.attached_collectors)
                    
            attached_collectors = {o.name:o.value for o in self.dequeue()}
            
            if attached_collectors:
                self.attached_collectors = merge(self.attached_collectors,attached_collectors)
            
            if 'batch' in attached_collectors:
                pbar.update(attached_collectors['batch'])
                
            if 'epoch' in attached_collectors:
                mbar.update(attached_collectors['epoch'])
                collector_values = {k:self.attached_collectors.get(k,None) for k in self.collector_keys}
                mbar.write([f'{l:.6f}' if isinstance(l, float) else str(l) for l in collector_values.values()], table=True)
            yield record

        attached_collectors = {o.name:o.value for o in self.dequeue()}
        if attached_collectors: self.attached_collectors = merge(self.attached_collectors,attached_collectors)

        collector_values = {k:self.attached_collectors.get(k,None) for k in self.collector_keys}
        mbar.write([f'{l:.6f}' if isinstance(l, float) else str(l) for l in collector_values.values()], table=True)

        pbar.on_iter_end()
        mbar.on_iter_end()
            

In [9]:
#|export
class RewardCollector(LogCollector):
    def __iter__(self):
        for q in self.main_queues: q.put(Record('reward',None))
        for steps in self.source_datapipe:
            if isinstance(steps,dp.DataChunk):
                for step in steps:
                    for q in self.main_queues: q.put(Record('reward',step.reward.detach().numpy()))
            else:
                for q in self.main_queues: q.put(Record('reward',steps.reward.detach().numpy()))
            yield steps

In [10]:
#|export
class EpocherCollector(dp.iter.IterDataPipe):
    def __init__(self,
            source_datapipe,
            epochs:int=0,
            logger_bases:List[LoggerBase]=None # `LoggerBase`s that we want to send metrics to
        ):
        self.source_datapipe = source_datapipe
        self.main_queues = [o.main_queue for o in logger_bases] if logger_bases is not None else None
        self.iteration_started = False
        self.epochs = epochs
        self.epoch = 0

    def __iter__(self): 
        if self.main_queues is not None and not self.iteration_started:
            for q in self.main_queues: q.put(Record('epoch',None))
            self.iteration_started = True
            
        for i in range(self.epochs): 
            self.epoch = i
            if self.main_queues is not None:
                for q in self.main_queues: q.put(Record('epoch',self.epoch))
            yield from self.source_datapipe
            
add_docs(
    EpocherCollector,
    """Tracks the number of epochs that the pipeline is currently on."""
)

In [11]:
#|export
class BatchCollector(dp.iter.IterDataPipe):
    def __init__(self,
            source_datapipe,
            logger_bases:List[LoggerBase], # `LoggerBase`s that we want to send metrics to
            batches:Optional[int]=None,
            # If `batches` is None, `BatchCollector` with try to find: `batch_on_pipe` instance
            # and try to grab a `batches` field from there.
            batch_on_pipe:dp.iter.IterDataPipe=None 
        ):
        self.source_datapipe = source_datapipe
        self.main_queues = [o.main_queue for o in logger_bases] if logger_bases is not None else None
        self.iteration_started = False
        self.batches = ifnone(
            batches,
            find_pipe_instance(self.source_datapipe,pipe_cls=batch_on_pipe).batches
        )
        self.batch = 0

    def __iter__(self): 
        if self.main_queues is not None and not self.iteration_started:
            for q in self.main_queues: q.put(Record('batch',None))
            self.iteration_started = True
            
        self.batch = 0
        for batch,record in enumerate(self.source_datapipe): 
            yield record
            self.batch = batch
            if self.main_queues is not None:
                for q in self.main_queues: q.put(Record('batch',batch))
                
add_docs(
    BatchCollector,
    """Tracks the number of batches that the pipeline is currently on."""
)

In [12]:
import pandas as pd
from fastrl.envs.gym import *

envs = ['CartPole-v1']*10

logger_base = ProgressBarLogger(batch_on_pipe=BatchCollector,epoch_on_pipe=EpocherCollector)

pipe = dp.map.Mapper(envs)
pipe = TypeTransformLoop(pipe,[GymTypeTransform])
pipe = dp.iter.MapToIterConverter(pipe)
pipe = dp.iter.InMemoryCacheHolder(pipe)
pipe = pipe.cycle(count=(18*len(envs))) 
pipe = EpocherCollector(pipe,epochs=5)
# Turn off the seed so that some envs end before others...
pipe = GymStepper(pipe,synchronized_reset=True)

pipe = RewardCollector(pipe,[logger_base])

# pipe = logger_base.connect_source_datapipe(pipe)
# pipe = add_cbs_to_pipes(pipe,L(cb))
# steps = list(pipe)
steps = list(pipe)

In [3]:
#|hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev import nbdev_export
    nbdev_export()